In [6]:
import pandas as pd

# V1
df = pd.read_csv('./data/Journals/FinancialJournal.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df.drop_duplicates(inplace=True)

df = df.groupby(['participantId', 'year', 'month', 'category'])['amount'].sum().reset_index()
df['date'] = (df['year'] * 100 + df['month']).astype(str)
df['date'] = pd.to_datetime(df['date'], format='%Y%m')
df = df.set_index('date').groupby(['participantId','category'])['amount'].resample('MS').asfreq().reset_index()

bussiness_area = ['Education', 'Food', 'Recreation', 'Shelter']
df = df.query('category in @bussiness_area').reset_index(drop=True)
df['amount'] = df['amount'].abs()
df.to_csv('./data/v1.csv', index=False)
# pd.pivot_table(df, index=['participantId', 'year', 'month'], columns=['category'], values=['amount'], aggfunc='sum')


In [8]:
df

,participantId,category,date,amount
0,0,Education,2022-03-01,38.005380
1,0,Education,2022-04-01,38.005380
2,0,Education,2022-05-01,38.005380
3,0,Education,2022-06-01,38.005380
4,0,Education,2022-07-01,38.005380
...,...,...,...,...
42201,1010,Shelter,2023-01-01,465.352332
42202,1010,Shelter,2023-02-01,465.352332
42203,1010,Shelter,2023-03-01,465.352332
42204,1010,Shelter,2023-04-01,465.352332


In [5]:
import pandas as pd

# V1
df = pd.read_csv('./data/Journals/FinancialJournal.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df.drop_duplicates(inplace=True)

df = df.groupby(['year', 'month', 'category'])['amount'].sum().reset_index()
df['date'] = (df['year'] * 100 + df['month']).astype(str)
df['date'] = pd.to_datetime(df['date'], format='%Y%m')
df = df.set_index('date').groupby('category')['amount'].resample('MS').asfreq().reset_index()


In [1]:
# V2
import os
import pandas as pd

res = pd.DataFrame()
for f in os.listdir('data/Activity_Logs/'):
    if not f.endswith('csv'):
        continue
    else:
        df = pd.read_csv(f'data/Activity_Logs/{f}')
        df = df.query('currentMode == "AtRestaurant"').reset_index(drop=True)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['year'] = df['timestamp'].dt.year
        df['month'] = df['timestamp'].dt.month

        tmp = df.groupby(['participantId', 'year', 'month'])['timestamp'].count().reset_index(name='hour')
        res = pd.concat([res, tmp], ignore_index=True)

res = res.groupby(['participantId', 'year', 'month'])['hour'].sum().reset_index()  # this is popularity
res['hour'] = res['hour'] * 5 / 60
res.to_csv('v2_pop.csv', index=False)

df = pd.read_csv('./data/Journals/FinancialJournal.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df.drop_duplicates(inplace=True)

# TODO
tmp_list = res['participantId'].unique()
res2 = df.query('participantId in @tmp_list and category == "Wage"').groupby(
    ['participantId', 'year', 'month'])['amount'].mean().reset_index()
res2.to_csv('V2_salary.csv', index=False)

In [28]:
# V3
df = pd.read_csv('./data/Journals/FinancialJournal.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df.drop_duplicates(inplace=True)

expense_list = ['Shelter', 'Education', 'Food', 'Recreation']
expense_res = df.query('category in @expense_list').groupby(
    ['participantId', 'year', 'month', 'category'], as_index=False)['amount'].mean().groupby(
        ['participantId', 'year', 'month'])['amount'].sum()
wage_res = df.query('category == "Wage"').groupby(
    ['participantId', 'year', 'month'], as_index=False
)['amount'].mean()

In [ ]:
# V4
